### Importér HoldetScraper-klassen

In [1]:
import pandas as pd
from holdetdk_scraper import HoldetScraper

### Lav en scraper instans.

In [2]:
scraper = HoldetScraper()

### Print en liste over aktive spil.

In [3]:
scraper.active_games

['Bundes Manager Efterår 2022',
 'EM Manager 2022',
 'Golf Manager 2022',
 'Motor Manager 2022',
 'Premier Manager Efterår 2022',
 'Super Manager Efterår 2022',
 'Tour Manager 2022',
 'Tourspillet 2022']

### Træk tabel og hold for Top X i præmiepuljen for den aktive runde i det valgte spil.
### Vær opmærksom på at scraperen trækker hold fra den aktive runde uanset hvilken runde der trækkes tabel for.

In [4]:
game = 'Premier Manager Efterår 2022' # - vælg evt. selv et andet spil af de aktive spil
round = 0 # træk for den aktive runde
top = 100
table, teams = scraper.get_table_and_teams(game=game, round=round, top=top)

Premier Manager Efterår 2022, Runde 1: Henter tabel for Top 100 i præmiepuljen: 100%|██████████| 5/5 [00:03<00:00,  1.60it/s]
Henter hold: 100%|██████████| 100/100 [00:59<00:00,  1.68it/s]


In [5]:
popularity = teams.groupby(['SpillerNavn', 'SpillerPosition']).size().reset_index(name='Antal')
popularity['Popularitet'] = (popularity['Antal']/len(table))
popularity.drop(['Antal'], axis = 1, inplace = True)

In [6]:
captain = teams[teams['SpillerKaptajn']].groupby(['SpillerNavn', 'SpillerPosition']).size().reset_index(name='Antal')
captain['Anfører'] = (captain['Antal']/len(table))
captain.drop(['Antal'], axis = 1, inplace = True)

In [8]:
final = popularity.merge(captain, how = 'left', left_on = ['SpillerNavn','SpillerPosition'], right_on = ['SpillerNavn','SpillerPosition'])
final['Anfører'] = final['Anfører'].fillna(0)
final.sort_values(by=['Popularitet', 'Anfører'], inplace=True, ascending=False)

### Gem tabel og hold i Excel-fil i Output-mappen.

In [ ]:
output_path = f'Output/{game}, Runde {round}, Top {len(table)}.xlsx'
with pd.ExcelWriter(output_path) as writer:  
    table.to_excel(writer, sheet_name='Tabel', index=False)
    teams.to_excel(writer, sheet_name='Hold', index=False)
    final.to_excel(writer, sheet_name='Popularitet', index=False)